# **Neural Style Transfer**
![](https://raw.githubusercontent.com/rohit9650/ML/master/Neural%20Style%20Transfer/3.png)



## By Rohit Singh



#*Motivation*



1.   Prisma
2.   Representation Learning
3.   A Neural Algorithm of Artistic Style



#*Overview*
![alt text](https://raw.githubusercontent.com/rohit9650/ML/master/Neural%20Style%20Transfer/4.png)

Neural style transfer consists of applying the style of a reference image to a target
image while conserving the content of the target image.

In this context, 
**style** *essentially means textures, colors, and visual patterns in the image, at
various spatial scales*; 
and the** content** *is the higher-level macrostructure of the image.*

For instance, blue-and-yellow circular brushstrokes are considered to be the style in (using Starry Night by Vincent Van Gogh), and the buildings in the photograph are considered to be the content.


#*Idea*

The key notion behind implementing style transfer is the same idea that’s central
to all deep-learning algorithms: you define a loss function to specify what you want to
achieve, and you minimize this loss.


##So what's the loss ?

####conserving the content of the original image while adopting the style of the reference image.



> loss = distance(style(reference_image) - style(generated_image)) + 
>> distance(content(original_image) - content(generated_image))


##Need a neural network

>we'll use VGG19


#Approach 
Neural style transfer can be implemented using any pretrained convnet. Here, you’ll
use the VGG19 network used by Gatys et al. VGG19 is a simple variant of the VGG16 net-
work introduced in chapter 5, with three more convolutional layers.
This is the general process:


1.   Set up a network that computes VGG19 layer activations for the style-reference
image, the target image, and the generated image at the same time.
2.   Use the layer activations computed over these three images to define the loss
function described earlier, which you’ll minimize in order to achieve style
transfer.
3.    Set up a gradient-descent process to minimize this loss function.











In [53]:
'''
Keras implementation of the original 2015 neural style transfer
algorithm.
Neural style transfer can be implemented using any pretrained convnet. 
Here, we’ll use the VGG19 network used by Gatys et al. 
VGG19 is a simple variant of the VGG16 network.
'''
! wget -O target.jpeg https://raw.githubusercontent.com/rohit9650/ML/master/Neural%20Style%20Transfer/my.jpg 
! wget -O style.jpeg https://images.unsplash.com/photo-1523541976599-15991b00a4a4?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=667&q=80

--2019-03-08 18:40:43--  https://raw.githubusercontent.com/rohit9650/ML/master/Neural%20Style%20Transfer/my.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75020 (73K) [image/jpeg]
Saving to: ‘target.jpeg’

target.jpeg         100%[===================>]  73.26K  --.-KB/s    in 0.02s   

2019-03-08 18:40:43 (3.51 MB/s) - ‘target.jpeg’ saved [75020/75020]

--2019-03-08 18:40:45--  https://images.unsplash.com/photo-1523541976599-15991b00a4a4?ixlib=rb-1.2.1
Resolving images.unsplash.com (images.unsplash.com)... 151.101.2.208, 151.101.66.208, 151.101.130.208, ...
Connecting to images.unsplash.com (images.unsplash.com)|151.101.2.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4211744 (4.0M) [image/jpeg]
Saving to: ‘style.jpeg’

style.j

In [0]:
# Defining initial variables

from keras.preprocessing.image import load_img, img_to_array

target_image_path = 'target.jpeg'
style_reference_image_path = 'style.jpeg'

width, height = load_img(target_image_path).size

img_height = 400
img_width = int(width * img_height / height)

#Pre-processing and Deprocessing

##pre-processing
>Don't need much, vgg19 gonna handle it.

##Deprocessing
> vgg19 has zero centered the image, so need to add mean of each dimension 
> also it has changed the image to BGR.

In [0]:
# Auxiliary functions

import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
  img = load_img(image_path, target_size=(img_height, img_width))
  img = img_to_array(img)
  img = np.expand_dims(img,axis=0)
  img = vgg19.preprocess_input(img)
  
  return img

def deprocess_img(x):
  '''
  Zero-centering by removing the mean pixel value
  from ImageNet. This reverses a transformation
  done by vgg19.preprocess_input.
  
  Converts images from 'BGR' to 'RGB'.
  This is also part of the reversal of
  vgg19.preprocess_input.
  '''
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  
  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype('uint8')
  
  return x

#Set up VGG19

It takes as input a batch of three images: 
1.    the style-reference image
2.    the target image and 
3.    a placeholder that will contain the generated image.

A placeholder is a symbolic tensor, the values of which are provided externally
via Numpy arrays. The style-reference and target image are static and thus defined
using K.constant , whereas the values contained in the placeholder of the generated
image will change over time

In [56]:
# Loading the pretrained VGG19 network and applying it to the three images

from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(
    style_reference_image_path))

combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image, 
                             style_reference_image, 
                             combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor, 
                    weights='imagenet', 
                    include_top=False)

print('Model Loaded.')

Model Loaded.


#Loss Function

##*Content Loss*
![alt text](https://raw.githubusercontent.com/rohit9650/ML/master/Neural%20Style%20Transfer/content_loss.png)

##*Style Loss*

 while calculating the style loss we will consider feature representation of many convolution layers from shallow to deeper layers of the model.
 
 ### *But Wait !!!*
 Unlike content loss we can’t just find the difference in activation units.
 
 ### *What Should we do ?*
 What we need is a way to find the correlation between these activations across different channels of the same layer and to do this we need something called as the Gram Matrix.
 
 ### *Gram Matrix*
 ![alt text](https://cdn-images-1.medium.com/max/800/0*p0RFEgFg_zP-J6kD)
 ![alt text](https://cdn-images-1.medium.com/max/800/0*u4U60W2sblmoCtv1)
 
Each element of this gram matrix contains correlation measure of all the channels with respect to each other.

### *how do we use this computed Gram matrix G to calculate the style loss*
Let’s denote the gram matrix of style image of layer L as GM[L] (S) and gram matrix of generated image of same layer as GM[L] (G). Both the gram matrix were computed from the same layer hence using the same number of channel leading it to be a matrix of size ch x ch, Now if we find sum of square difference or L2_norm of element subtraction of these two matrices and try to minimize it, Then this will eventually lead to minimizing the difference between the style of style image and the generated image. 

![alt text](https://cdn-images-1.medium.com/max/800/1*IoozR3xGzaSqtEqGEKcWMQ.jpeg)

In the above equation, *N* subscript *l* represents the number of channel in the feature-map/output of layer *l* and *M* subscript *l* represents the height x width of the feature-map/output of layer *l*.

## *We can add total variation loss*
It operates on the pixels of the generated combination image. It encourages spatial continuity in
the generated image, thus avoiding overly pixelated results. We can interpret it as a
regularization loss.

In [0]:
# Content loss

def content_loss(base, combination):
  return K.sum(K.square(combination - base))

# Style loss

def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
  gram = K.dot(features, K.transpose(features))
  
  return gram

def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height*img_width
  
  return K.sum(K.square(S-C)) / (4. * (channels ** 2) * (size ** 2))

# Total variation loss

def total_variation_loss(x):
  '''
  It operates on the pixels of the generated combination image. 
  It encourages spatial continuity in the generated image, thus avoiding 
  overly pixelated results. You can interpret it as a regularization loss.
  '''
  a = K.square(
      x[:, :img_height-1, :img_width-1, :] - 
      x[:, 1:, :img_width-1, :])
  b = K.square(
      x[:, :img_height-1, :img_width-1, :] - 
      x[:, :img_height-1, 1:, :])
  
  return K.sum(K.pow(a+b, 1.25))

In [0]:
# Defining the final loss that is needed to be minimized

outputs_dict = dict([(layer.name, layer.output) 
                     for layer in model.layers])

content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1', 
                'block3_conv1', 
                'block4_conv1',
                'block5_conv1']

total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.variable(0.)

layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features,
                                            combination_features)

for layer_name in style_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_reference_features, combination_features)
  loss = loss + (style_weight / len(style_layers)) * sl

loss = loss + total_variation_weight * total_variation_loss(
    combination_image)
  


# Setting up the gradient-descent process

Finally, we’ll set up the gradient-descent process. In the original 
Gatys et al. paper, optimization is performed using the [L-BFGS algorithm](https://en.wikipedia.org/wiki/Limited-memory_BFGS), 
so that’s what we’ll use here.
The L-BFGS algorithm comes packaged with SciPy, but there are two slight
limitations with the SciPy implementation:

* It requires that you pass the value of the loss function and the value 
  of the gradients as two separate functions.
* It can only be applied to flat vectors, whereas we have a 3D img array.

It would be inefficient to compute the value of the loss function and 
the value of the gradients independently, because doing so would lead
to a lot of redundant computation between the two; the process would be 
almost twice as slow as computing them jointly. To bypass this, we’ll set
up a Python class named Evaluator that computes both the loss value 
and the gradients value at once, returns the loss value when called
the first time, and caches the gradients for the next call.


In [0]:


grads = K.gradients(loss, combination_image)[0]

fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
  
  def __init__(self):
    self.loss_value = None
    self.grads_values = None
  
  def loss(self, x):
    assert self.loss_value is None
    
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads([x])
    
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    
    self.loss_value = loss_value
    self.grad_values = grad_values
    
    return self.loss_value
  
  def grads(self, x):
    assert self.loss_value is not None
    
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_values = None
    
    return grad_values

evaluator = Evaluator()

# Style-transfer loop

Finally, we can run the gradient-ascent process using SciPy’s  L-BFGS 
algorithm, saving the current generated image at each iteration of the 
algorithm (a single iteration represents 20 steps of gradient ascent).


In [60]:


from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'my_result'
iterations = 20

x = preprocess_image(target_image_path)
x = x.flatten()

for i in range(iterations):
  print('start of iteration : ', i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, 
                                   x,
                                   fprime=evaluator.grads,
                                   maxfun=20)
  print('current loss value : ', min_val)
  img = x.copy().reshape((img_height, img_width, 3))
  img = deprocess_img(img)
  fname = result_prefix + '_at_iteration_%d.png' % i
  imsave(fname, img)
  
  print('image saved as : ', fname)
  end_time = time.time()
  print('Iteration %d completed in %ds' % (i, end_time - start_time))

start of iteration :  0
current loss value :  2274801700.0
image saved as :  my_result_at_iteration_0.png
Iteration 0 completed in 10s
start of iteration :  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


current loss value :  784558100.0
image saved as :  my_result_at_iteration_1.png
Iteration 1 completed in 9s
start of iteration :  2
current loss value :  477150200.0
image saved as :  my_result_at_iteration_2.png
Iteration 2 completed in 9s
start of iteration :  3
current loss value :  374312770.0
image saved as :  my_result_at_iteration_3.png
Iteration 3 completed in 9s
start of iteration :  4
current loss value :  294365470.0
image saved as :  my_result_at_iteration_4.png
Iteration 4 completed in 9s
start of iteration :  5
current loss value :  256049650.0
image saved as :  my_result_at_iteration_5.png
Iteration 5 completed in 9s
start of iteration :  6
current loss value :  227453420.0
image saved as :  my_result_at_iteration_6.png
Iteration 6 completed in 9s
start of iteration :  7
current loss value :  209179090.0
image saved as :  my_result_at_iteration_7.png
Iteration 7 completed in 9s
start of iteration :  8
current loss value :  196532460.0
image saved as :  my_result_at_iter

#Resources : 

1.    [A Neural Algorithm of Artistic Style](https://arxiv.org/pdf/1508.06576.pdf)
2.    [TowardsDataScience Article](https://towardsdatascience.com/neural-style-transfer-tutorial-part-1-f5cd3315fa7f)
3.    [Deep Learning with Python By Francois Chollet](https://www.amazon.in/Deep-Learning-Python-Francois-Chollet/dp/1617294438)


![](https://raw.githubusercontent.com/rohit9650/ML/master/Neural%20Style%20Transfer/my_1.png)
#**THANK YOU**